In [20]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
full_df = pd.read_csv(os.path.join('datasets', 'JRE_kevinHart_25052020.csv'))
speaker_series = full_df.groupby('speaker')['text'].apply(lambda x: ''.join(x))

### COSINE SIMILARITY

In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

import torch.nn.functional as F


[nltk_data] Downloading package stopwords to /home/guayo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/guayo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
#Get random sentences from both. Execute until satisfied with choice
random_sent_joe = np.random.choice(full_df[full_df['speaker'] == 'JOEROGAN']['text'].to_list())
random_sent_kevin = np.random.choice(full_df[full_df['speaker'] == 'KEVINHART']['text'].to_list())
print("JOEROGAN:", random_sent_joe)
print("KEVINHART:", random_sent_kevin)

JOEROGAN:  That's amazing. Framed in my fucking house. I hope Eddie comes back. I know he's been talking about it. He's been talking about it. Did you see that one thing that he did? There was some sort of an award show where he came up and talked on the podium and he was doing material and he was talking about them taking away Bill Cosby's degrees. No, you never seen it? No, Jamie. Find it. Find it.
KEVINHART:  He knows where all these after hours places are. You go behind an alley, you knock on a door, they open it up.


In [24]:
# Preprocessing (assuming the model was trained on lowercased text)
def preprocess(text):
    return [word for word in word_tokenize(text) if word.isalpha() and word not in stop_words]

Get similarity with Word2Vec

In [39]:
from gensim.models import Word2Vec
from gensim import matutils

# Load your pre-trained Word2Vec model
model = Word2Vec.load('podcast_Word2Vec')

In [66]:
# Convert text to vectors
def text_to_vector(text):
    words = preprocess(text)
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

corpus_joe = text_to_vector(speaker_series["JOEROGAN"])
corpus_kevin = text_to_vector(speaker_series["KEVINHART"])
vec_joe = text_to_vector(random_sent_joe)
vec_kevin = text_to_vector(random_sent_kevin)

from sklearn.metrics.pairwise import cosine_similarity

# OUTPUT SHOULD BE --> JOE ROGAN SAID THIS
corpus_joe_sim = cosine_similarity(corpus_joe.reshape(1, -1), vec_joe.reshape(1, -1))
corpus_kevin_sim = cosine_similarity(corpus_kevin.reshape(1, -1), vec_joe.reshape(1, -1))

if corpus_joe_sim > corpus_kevin_sim:
    print('JOE ROGAN SAID THIS')
else:
    print('KEVIN HART SAID THIS')

# OUTPUT SHOULD BE --> KEVIN HART SAID THIS
corpus_joe_sim = cosine_similarity(corpus_joe.reshape(1, -1), vec_kevin.reshape(1, -1))
corpus_kevin_sim = cosine_similarity(corpus_kevin.reshape(1, -1), vec_kevin.reshape(1, -1))

if corpus_joe_sim > corpus_kevin_sim:
    print('JOE ROGAN SAID THIS')
else:
    print('KEVIN HART SAID THIS')


KEVIN HART SAID THIS
KEVIN HART SAID THIS


Get the similarity with ELMo

In [26]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

elmo = Elmo(options_file, weight_file, 1, dropout=0)

In [31]:
def get_elmo_embeddings(words):
    character_ids = batch_to_ids(words)
    embeddings = elmo(character_ids)
    return embeddings['elmo_representations'][0]

random_sent_joe_embedded = get_elmo_embeddings(random_sent_joe)
random_sent_kevin_embedded = get_elmo_embeddings(random_sent_kevin)
corpus_joe_embedded = get_elmo_embeddings(corpus_joe)
corpus_kevin_embedded = get_elmo_embeddings(corpus_kevin)

def average_embedding(sentence_embedding):
    return sentence_embedding.mean(dim=0)

similarities = [cosine_similarity(corpus_joe, vec_joe), cosine_similarity(corpus_kevin, vec_kevin)]
print(similarities)

similarities[
    cosine_similarity(average_embedding(corpus_joe_embedded), average_embedding(random_sent_joe_embedded), dim=0),
    cosine_similarity(average_embedding(corpus_kevin_embedded), average_embedding(random_sent_joe_embedded), dim=0)
]

print(f"La similitud del coseno entre las dos oraciones es: {[s.item() for s in similarities]}")

TypeError: 'numpy.float32' object is not iterable